Hosting of my chess program on lichess, making use of the free Google Colab GPUs.

In [ ]:
!pip install chess

import numpy as np
import torch
import torch.nn as nn
import chess
import re

from google.colab import drive, userdata
drive.mount('/content/drive', force_remount=True)

# configuring device
try:
    device = xm.xla_device()
    print("Running on the TPU")
except:
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
        print('Running on the GPU')
        torch.cuda.synchronize()
    else:
        device = torch.device('cpu')
        print('Running on the CPU')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148497 sha256=04906b810f3abff1ba3f55f1adf4ae19ca78a4f788f06e25a7192774a56abe09
  Stored in directory: /root/.cache/pip/wheels/f0/3f/76/8783033e8524d407e1bebaf72fdd3f3eba27e0c030e92bbd87
Successfully built chess
Mounted at /content/drive
Running on the GPU


In [ ]:
# Variable initialisation if needed.

In [ ]:
# Setup lichess-bot.
!git clone https://github.com/walterliu417/lichess-bot-parrot.git
!pip install -r "lichess-bot-parrot/requirements.txt"

import os
os.chdir("lichess-bot-parrot")

Cloning into 'lichess-bot-parrot'...
remote: Enumerating objects: 4953, done.
remote: Counting objects: 100% (534/534), done.
remote: Compressing objects: 100% (271/271), done.
remote: Total 4953 (delta 372), reused 281 (delta 263), pack-reused 4419 (from 5)
Receiving objects: 100% (4953/4953), 1.57 MiB | 3.85 MiB/s, done.
Resolving deltas: 100% (3261/3261), done.


In [ ]:
lichessapi = userdata.get("lichessapi")

config = f'''token: "{lichessapi}"    # Lichess OAuth2 Token.
url: "https://lichess.org/"        # Lichess base URL.

engine:                            # Engine settings.
  dir: "./engines/"                # Directory containing the engine. This can be an absolute path or one relative to lichess-bot/.
  name: "Parrot"              # Binary name of the engine to use.
  interpreter: "python"
# interpreter_options:
#   - "-jar"
  working_dir: ""                  # Directory where the chess engine will read and write files. If blank or missing, the current directory is used.
                                   # NOTE: If working_dir is set, the engine will look for files and directories relative to this directory, not where lichess-bot was launched. Absolute paths are unaffected.
  protocol: "homemade"                  # "uci", "xboard" or "homemade"
  ponder: false                     # Think on opponent's time.

  polyglot:
    enabled: false                 # Activate polyglot book.
    book:
      standard:                    # List of book file paths for variant standard.
        - engines/book1.bin
        - engines/book2.bin
#     atomic:                      # List of book file paths for variant atomic.
#       - engines/atomicbook1.bin
#       - engines/atomicbook2.bin
#     etc.
#     Use the same pattern for 'chess960', 'giveaway' (antichess), 'crazyhouse', 'horde', 'kingofthehill', 'racingkings' and '3check' as well.
    min_weight: 1                  # Does not select moves with weight below min_weight (min 0, max: 65535).
    selection: "weighted_random"   # Move selection is one of "weighted_random", "uniform_random" or "best_move" (but not below the min_weight in the 2nd and 3rd case).
    max_depth: 20                  # How many moves from the start to take from the book.

  draw_or_resign:
    resign_enabled: false          # Whether or not the bot should resign.
    resign_score: -1000            # If the score is less than or equal to this value, the bot resigns (in cp).
    resign_for_egtb_minus_two: true # If true the bot will resign in positions where the online_egtb returns a wdl of -2.
    resign_moves: 3                # How many moves in a row the score has to be below the resign value.
    offer_draw_enabled: true       # Whether or not the bot should offer/accept draw.
    offer_draw_score: 0            # If the absolute value of the score is less than or equal to this value, the bot offers/accepts draw (in cp).
    offer_draw_for_egtb_zero: true # If true the bot will offer/accept draw in positions where the online_egtb returns a wdl of 0.
    offer_draw_moves: 10           # How many moves in a row the absolute value of the score has to be below the draw value.
    offer_draw_pieces: 10          # Only if the pieces on board are less than or equal to this value, the bot offers/accepts draw.

  online_moves:
    max_out_of_book_moves: 10      # Stop using online opening books after they don't have a move for 'max_out_of_book_moves' positions. Doesn't apply to the online endgame tablebases.
    max_retries: 2                 # The maximum amount of retries when getting an online move.
    # max_depth: 10                # How many moves from the start to take from online books. Default is no limit.
    chessdb_book:
      enabled: false               # Whether or not to use chessdb book.
      min_time: 20                 # Minimum time (in seconds) to use chessdb book.
      move_quality: "good"         # One of "all", "good", "best".
      min_depth: 20                # Only for move_quality: "best".
    lichess_cloud_analysis:
      enabled: false               # Whether or not to use lichess cloud analysis.
      min_time: 20                 # Minimum time (in seconds) the bot must have to use cloud analysis.
      move_quality: "best"         # One of "good", "best".
      max_score_difference: 50     # Only for move_quality: "good". The maximum score difference (in cp) between the best move and the other moves.
      min_depth: 20
      min_knodes: 0
    lichess_opening_explorer:
      enabled: false
      min_time: 20
      source: "masters"            # One of "lichess", "masters", "player"
      player_name: ""              # The lichess username. Leave empty for the bot's username to be used. Used only when source is "player".
      sort: "winrate"              # One of "winrate", "games_played"
      min_games: 10                # Minimum number of times a move must have been played to be chosen.
    online_egtb:
      enabled: false               # Whether or not to enable online endgame tablebases.
      min_time: 20                 # Minimum time (in seconds) the bot must have to use online EGTBs.
      max_pieces: 7                # Maximum number of pieces on the board to use endgame tablebases.
      source: "lichess"            # One of "lichess", "chessdb".
      move_quality: "best"         # One of "best" or "suggest" (it takes all the moves with the same WDL and tells the engine to only consider these; will move instantly if there is only 1 "good" move).

  lichess_bot_tbs:                 # The tablebases list here will be read by lichess-bot, not the engine.
    syzygy:
      enabled: true               # Whether or not to use local syzygy endgame tablebases.
      paths:                       # Paths to Syzygy endgame tablebases.
        - "/content/drive/MyDrive/parrot/tablebase_5pc"
      max_pieces: 5                # Maximum number of pieces in the endgame tablebase.
      move_quality: "best"         # One of "best" or "suggest" (it takes all the moves with the same WDL and tells the engine to only consider these; will move instantly if there is only 1 "good" move).
    gaviota:
      enabled: false               # Whether or not to use local gaviota endgame tablebases.
      paths:
        - "engines/gaviota"
      max_pieces: 5
      min_dtm_to_consider_as_wdl_1: 120 # The minimum DTM to consider as syzygy WDL=1/-1. Set to 100 to disable.
      move_quality: "best"         # One of "best" or "suggest" (it takes all the moves with the same WDL and tells the engine to only consider these; will move instantly if there is only 1 "good" move).

# engine_options:                  # Any custom command line params to pass to the engine.
#   cpuct: 3.1

  homemade_options:
#   Hash: 256

  uci_options:                     # Arbitrary UCI options passed to the engine.
    Move Overhead: 100             # Increase if your bot flags games too often.
    Threads: 4                     # Max CPU threads the engine can use.
    Hash: 512                      # Max memory (in megabytes) the engine can allocate.
    SyzygyPath: "./syzygy/"        # Paths to Syzygy endgame tablebases that the engine reads.
    UCI_ShowWDL: true              # Show the chance of the engine winning.
#   go_commands:                   # Additional options to pass to the UCI go command.
#     nodes: 1                     # Search so many nodes only.
#     depth: 5                     # Search depth ply only.
#     movetime: 1000               # Integer. Search exactly movetime milliseconds.

# xboard_options:                  # Arbitrary XBoard options passed to the engine.
#   cores: "4"
#   memory: "4096"
#   egtpath:                       # Directory containing egtb (endgame tablabases), relative to this project. For 'xboard' engines.
#     gaviota: "Gaviota path"
#     nalimov: "Nalimov Path"
#     scorpio: "Scorpio Path"
#     syzygy: "Syzygy Path"
#   go_commands:                   # Additional options to pass to the XBoard go command.
#     depth: 5                     # Search depth ply only.
#     Do note that the go commands 'movetime' and 'nodes' are invalid and may cause bad time management for XBoard engines.

  silence_stderr: false            # Some engines (yes you, Leela) are very noisy.

abort_time: 30                     # Time to abort a game in seconds when there is no activity.
fake_think_time: false             # Artificially slow down the bot to pretend like it's thinking.
rate_limiting_delay: 0             # Time (in ms) to delay after sending a move to prevent "Too Many Requests" errors.
move_overhead: 2000                # Increase if your bot flags games too often.
max_takebacks_accepted: 0          # The number of times to allow an opponent to take back a move in a game.
quit_after_all_games_finish: false # If set to true, then pressing Ctrl-C to quit will only stop lichess-bot after all current games have finished.

correspondence:
  move_time: 60                    # Time in seconds to search in correspondence games.
  checkin_period: 300              # How often to check for opponent moves in correspondence games after disconnecting.
  disconnect_time: 150             # Time before disconnecting from a correspondence game.
  ponder: false                    # Ponder in correspondence games the bot is connected to.

challenge:                         # Incoming challenges.
  concurrency: 1                   # Number of games to play simultaneously.
  sort_by: "best"                  # Possible values: "best" and "first".
  preference: "none"               # Possible values: "none", "human", "bot".
  accept_bot: true                 # Accepts challenges coming from other bots.
  only_bot: false                  # Accept challenges by bots only.
  max_increment: 20                # Maximum amount of increment to accept a challenge in seconds. The max is 180. Set to 0 for no increment.
  min_increment: 0                 # Minimum amount of increment to accept a challenge in seconds.
  max_base: 10800                   # Maximum amount of base time to accept a challenge in seconds. The max is 10800 (3 hours).
  min_base: 600                      # Minimum amount of base time to accept a challenge in seconds.
  max_days: 14                     # Maximum number of days per move to accept a challenge for a correspondence game.
                                   # Unlimited games can be accepted by removing this field or specifying .inf
  min_days: 1                      # Minimum number of days per move to accept a challenge for a correspondence game.
  variants:                        # Chess variants to accept (https://lichess.org/variant).
    - standard
#   - fromPosition
#   - antichess
#   - atomic
#   - chess960
#   - crazyhouse
#   - horde
#   - kingOfTheHill
#   - racingKings
#   - threeCheck
  time_controls:                   # Time controls to accept (bots are not allowed to play ultraBullet).
    - rapid
    - classical
#   - correspondence
  modes:                           # Game modes to accept.
    - casual                       # Unrated games.
    - rated                        # Rated games - must comment if the engine doesn't try to win.
# block_list:                      # List of users from which the challenges are always declined.
#   - user1
#   - user2
# allow_list:                      # List of users from which challenges are exclusively accepted, all others being declined. If empty, challenges from all users may be accepted.
#   - user3
#   - user4
# recent_bot_challenge_age: 60     # Maximum age of a bot challenge to be considered recent in seconds
# max_recent_bot_challenges: 2     # Maximum number of recent challenges that can be accepted from the same bot
  bullet_requires_increment: false # Require that bullet game challenges from bots have a non-zero increment
  max_simultaneous_games_per_user: 5  # Maximum number of simultaneous games with the same user

greeting:
  hello: "Hi! I'm {{me}}, a neural network based chess engine developed by YoMan417. Good luck!" # Message to send to opponent chat at the start of a game
  goodbye: "Good game!" # Message to send to opponent chat at the end of a game
  hello_spectators: "Hi! I'm {{me}}." # Message to send to spectator chat at the start of a game
  goodbye_spectators: "Thanks for watching!" # Message to send to spectator chat at the end of a game

# pgn_directory: "game_records"    # A directory where PGN-format records of the bot's games are kept
# pgn_file_grouping: "game"        # How to group games into files. Options are "game", "opponent", and "all"
                                   # "game" (default) - every game is written to a different file named "{{White name}} vs. {{Black name}} - {{lichess game ID}}.pgn"
                                   # "opponent" - every game with a given opponent is written to a file named "{{Bot name}} games vs. {{Opponent name}}.pgn"
                                   # "all" - every game is written to a single file named "{{Bot name}} games.pgn"

matchmaking:
  allow_matchmaking: false         # Set it to 'true' to challenge other bots.
  allow_during_games: false        # Set it to 'true' to create challenges during long games.
  challenge_variant: "random"      # If set to 'random', the bot will choose one variant from the variants enabled in 'challenge.variants'.
  challenge_timeout: 30            # Create a challenge after being idle for 'challenge_timeout' minutes. The minimum is 1 minute.
  challenge_initial_time:          # Initial time in seconds of the challenge (to be chosen at random).
    - 60
    - 180
  challenge_increment:             # Increment in seconds of the challenge (to be chosen at random).
    - 1
    - 2
#  challenge_days:                 # Days for correspondence challenge (to be chosen at random).
#    - 1
#    - 2
# opponent_min_rating: 600         # Opponents rating should be above this value (600 is the minimum rating in lichess).
# opponent_max_rating: 4000        # Opponents rating should be below this value (4000 is the maximum rating in lichess).
  opponent_rating_difference: 3000  # The maximum difference in rating between the bot's rating and opponent's rating.
  rating_preference: "none"        # One of "none", "high", "low".
  opponent_allow_tos_violation: false # Set to 'true' to allow challenging bots that violated the Lichess Terms of Service.
  challenge_mode: "random"         # Set it to the mode in which challenges are sent. Possible options are 'casual', 'rated' and 'random'.
  challenge_filter: none           # If a bot declines a challenge, do not issue a similar challenge to that bot. Possible options are 'none', 'coarse', and 'fine'.
# block_list:                      # The list of bots that will not be challenged
#   - user1
#   - user2
  include_challenge_block_list: false  # Do not challenge bots in the challenge: block_list in addition to the matchmaking block list.

# overrides:                       # List of overrides for the matchmaking specifications above. When a challenge is created, either the default specification above or one of the overrides will be randomly chosen.
#   bullet_only_horde:             # Name of the override. Can be anything as long as each override has a unique name ("bullet_only_horde" and "easy_chess960" in these examples).
#     challenge_variant: "horde"   # List of options to override. Only the options mentioned will change when making the challenge. The rest will follow the default matchmaking options above.
#     challenge_initial_time:
#       - 1
#       - 2
#     challenge_increment:
#       - 0
#       - 1
#
#   easy_chess960:
#     challenge_variant: "chess960"
#     opponent_min_rating: 400
#     opponent_max_rating: 1200
#     opponent_rating_difference:
#     challenge_mode: casual
#
#   no_pressure_correspondence:
#     challenge_initial_time:
#     challenge_increment:
#     challenge_days:
#       - 2
#       - 3
#     challenge_mode: casual
#
# The following configurations cannot be overridden: allow_matchmaking, challenge_timeout, challenge_filter and block_list.'''

with open("config.yml", "w") as file:
    file.write(config)

In [ ]:
!python lichess-bot.py


    .   _/|
    .  // o\
    .  || ._)  lichess-bot 2025.2.3.2 on Linux 6.1.85+
    .  //__\
    .  )___(   Play on Lichess with a bot
    
Checking engine configuration ...
Running on the GPU
SimpleModel(
  (conv_net): Sequential(
    (Conv 1): Conv2d(1, 100, kernel_size=(2, 2), stride=(1, 1))
    (Batchnorm 1): BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Conv activation): LeakyReLU(negative_slope=0.01)
    (Conv 2): Conv2d(100, 250, kernel_size=(2, 2), stride=(1, 1))
    (Batchnorm 2): BatchNorm2d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Conv activation 2): LeakyReLU(negative_slope=0.01)
    (Conv 3): Conv2d(250, 400, kernel_size=(3, 3), stride=(1, 1))
    (Batchnorm 3): BatchNorm2d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Conv activation 3): LeakyReLU(negative_slope=0.01)
    (Conv 4): Conv2d(400, 700, kernel_size=(4, 4), stride=(1, 1))
    (Batchnorm 4): BatchNorm2d(700, eps

KeyboardInterrupt: 